In [ ]:
!pip install -q sentence-transformers datasets faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, util, LoggingHandler
from torch.utils.data import DataLoader
from datetime import datetime
import torch, torch.nn.functional as F
import os, logging, tarfile, pickle
import faiss


In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
logging.basicConfig(format="%(asctime)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

In [ ]:
model_name = 'distilbert-base-uncased'
max_seq_length = 300
batch_size = 16
epochs = 1
pooling = "mean"
data_folder = "msmarco-data"
max_records = 30  #

In [ ]:
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
corpus = {}
collection_filepath = os.path.join(data_folder, "collection.tsv")
if not os.path.exists(collection_filepath):
    tar_path = os.path.join(data_folder, "collection.tar.gz")
    util.http_get("https://msmarco.z22.web.core.windows.net/msmarcoranking/collection.tar.gz", tar_path)
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(path=data_folder)
with open(collection_filepath, encoding="utf8") as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        corpus[int(pid)] = passage

  0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [ ]:
queries = {}
queries_filepath = os.path.join(data_folder, "queries.train.tsv")
if not os.path.exists(queries_filepath):
    tar_path = os.path.join(data_folder, "queries.tar.gz")
    util.http_get("https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz", tar_path)
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(path=data_folder)
with open(queries_filepath, encoding="utf8") as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[int(qid)] = query

  0%|          | 0.00/18.9M [00:00<?, ?B/s]

In [ ]:
qrels_url = "https://msmarco.z22.web.core.windows.net/msmarcoranking/qrels.train.tsv"
qrels_path = os.path.join(data_folder, "qrels.train.tsv")
if not os.path.exists(qrels_path):
    util.http_get(qrels_url, qrels_path)

  0%|          | 0.00/10.6M [00:00<?, ?B/s]

In [ ]:
seen = set()
train_examples = []
with open(qrels_path, encoding='utf8') as f:
    for line in f:
        if len(train_examples) >= max_records:
            break
        qid, _, pid, rel = line.strip().split('\t')
        qid = int(qid)
        pid = int(pid)
        if rel == '1' and (qid, pid) not in seen:
            seen.add((qid, pid))
            if qid in queries and pid in corpus:
                train_examples.append(InputExample(texts=[queries[qid], corpus[pid]]))

print(f" Loaded {len(train_examples)} (query, positive) training pairs.")

 Loaded 30 (query, positive) training pairs.


In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)

model_save_path = f'output/bi-encoder-30-msmarco-{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}'
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=10,
    use_amp=True,
    optimizer_params={'lr': 2e-5},
    show_progress_bar=True
)
model.save(model_save_path)
print(f"\n Model saved to: {model_save_path}")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss



 Model saved to: output/bi-encoder-30-msmarco-2025-04-17_14-00-00


In [ ]:
unique_passages = list({ex.texts[1] for ex in train_examples})
chunk_embeddings = model.encode(unique_passages, convert_to_tensor=True, show_progress_bar=True)
chunk_embeddings_np = chunk_embeddings.cpu().detach().numpy().astype("float32")
faiss.normalize_L2(chunk_embeddings_np)
index = faiss.IndexFlatIP(chunk_embeddings_np.shape[1])
index.add(chunk_embeddings_np)
faiss.write_index(index, "faiss_index_30_chunks.index")
with open("faiss_passages_30.pkl", "wb") as f:
    pickle.dump(unique_passages, f)

print(" FAISS index and passage text mapping saved.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 FAISS index and passage text mapping saved.


In [ ]:
index = faiss.read_index("faiss_index_30_chunks.index")
with open("faiss_passages_30.pkl", "rb") as f:
    passages = pickle.load(f)

query = "What is the role of mitochondria?"
query_embedding = model.encode(query, convert_to_numpy=True).astype("float32")
faiss.normalize_L2(query_embedding.reshape(1, -1))

top_k = 3
D, I = index.search(query_embedding.reshape(1, -1), top_k)
print(f"\n Top {top_k} passages for query: '{query}'\n")
for score, idx in zip(D[0], I[0]):
    print(f"Score: {score:.4f}")
    print(f"Chunk: {passages[idx]}")
    print("-" * 50)



 Top 3 passages for query: 'What is the role of mitochondria?'

Score: 0.7618
Chunk: The rhetoric of mental health considers how language functions in the production of knowledge on topics such as mental and psychological disorders, chemical imbalances in the brain, and variations on what are considered normal mental faculties.
--------------------------------------------------
Score: 0.7376
Chunk: Mushrooms are a good source of B vitamins, including riboflavin, niacin, and pantothenic acid, which help to provide energy by breaking down proteins, fats and carbohydrates2. B vitamins also play an important role in the nervous system.
--------------------------------------------------
Score: 0.7336
Chunk: Lyme disease causes depression, which can cause mood swings. Additionally, the pain associated with Lyme is enough to cause patients to become irritable. Lyme disease is a systemic infection that can affect all systems of the human body. This disease is a bacterial infection transmitted